In [29]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords


In [30]:
#loading the data

data=pd.read_csv('data/imdb_labelled.txt' , delimiter='\t', names=['review', 'sentiment'])

In [31]:
data.head()


,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     748 non-null    object
 1   sentiment  748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [33]:
data['sentiment'].value_counts()

1    386
0    362
Name: sentiment, dtype: int64

In [74]:
def text_filtering(text):
    #stop_words = list(stopwords.words('english'))
    stop_words=[]
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    for word in text.split(" "):
        if word in stop_words:
            text=text.replace(word,'')
    text = text.rstrip() 
    regular_expression = '[^a-zA-z\s]'
    text = re.sub(regular_expression, '', text)
    return text
text_filtering("movie is not good")

'movie is not good'

In [75]:
train_data = data[:int(0.7*len(data))]
test_and_dev_data = data[int(0.7*len(data)):]


dev_data=test_and_dev_data[:int(0.5 * len(test_and_dev_data))]
test_data=test_and_dev_data[int(0.5 * len(test_and_dev_data)):]

print(train_data.info())

print()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     523 non-null    object
 1   sentiment  523 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.3+ KB
None



In [76]:
print(test_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 635 to 747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     113 non-null    object
 1   sentiment  113 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.9+ KB
None


In [77]:
print(dev_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 523 to 634
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     112 non-null    object
 1   sentiment  112 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.9+ KB
None


In [78]:
train_data['sentiment'].value_counts()

0    292
1    231
Name: sentiment, dtype: int64

In [79]:
test_data['sentiment'].value_counts()

1    72
0    41
Name: sentiment, dtype: int64

In [80]:
dev_data['sentiment'].value_counts()

1    83
0    29
Name: sentiment, dtype: int64

In [81]:
X_train = train_data['review']
y_train = train_data['sentiment']
# print(X_train)
# print(y_train)

X_dev = dev_data['review']
y_dev = dev_data['sentiment']
# print(X_dev)
# print(y_dev)

X_test = test_data['review']
y_test = test_data['sentiment']
# print(X_test)
# print(y_test)

In [82]:

def cal_prior_probs(y_train):
    prior_probs = []
    count_0 = sum(y_train == 0)
    prior_probs.append(count_0 / y_train.size)  
    count_1 = sum(y_train == 1)
    prior_probs.append(count_1 / y_train.size)    
    return prior_probs
prior_probs=cal_prior_probs(y_train)
print("Negative review class' prior probability--",prior_probs[0] )
print("Positive review class' prior probability--",prior_probs[1] )

Negative review class' prior probability-- 0.5583173996175909
Positive review class' prior probability-- 0.4416826003824092


In [83]:
def build_word_dict(data):
    word_freq = {}
    all_words=[]
    for sentence in data:
            words=[]
            sentence = text_filtering(sentence)
            words=sentence.split(' ')
            for word in set(words) :
                word = word.lower()
                if word not in word_freq.keys():
                    word_freq[word] = 1
                    all_words.append(word)
                else:
                    word_freq[word] += 1
  
    return word_freq, all_words

    
       
allwords_freq,allwords_train=build_word_dict(X_train)
print("Total words--",len(allwords_train))


train_pos_reviews=train_data.loc[train_data.sentiment==1].review
pos_words_freq,pos_words=build_word_dict(train_pos_reviews)
print("Positive words--",len(pos_words))


train_neg_reviews=train_data.loc[train_data.sentiment==0].review
neg_words_freq,neg_words=build_word_dict(train_neg_reviews)
print("Negative words--",len(neg_words))

Total words-- 2482
Positive words-- 1267
Negative words-- 1748


In [84]:
allwords_freq.pop('', None)

for word in allwords_freq:
    print(word,"--",allwords_freq[word],"    probability=",allwords_freq[word]/len(X_train))

slowmoving -- 1     probability= 0.0019120458891013384
a -- 194     probability= 0.37093690248565964
drifting -- 1     probability= 0.0019120458891013384
very -- 35     probability= 0.06692160611854685
man -- 8     probability= 0.015296367112810707
movie -- 92     probability= 0.17590822179732313
about -- 24     probability= 0.045889101338432124
aimless -- 1     probability= 0.0019120458891013384
distressed -- 1     probability= 0.0019120458891013384
young -- 3     probability= 0.0057361376673040155
characters -- 19     probability= 0.036328871892925434
half -- 6     probability= 0.011472275334608031
of -- 160     probability= 0.30592734225621415
was -- 98     probability= 0.18738049713193117
audience -- 4     probability= 0.0076481835564053535
nearly -- 1     probability= 0.0019120458891013384
or -- 24     probability= 0.045889101338432124
lost -- 3     probability= 0.0057361376673040155
the -- 309     probability= 0.5908221797323135
walked -- 1     probability= 0.0019120458891013384


again -- 9     probability= 0.017208413001912046
horrible -- 4     probability= 0.0076481835564053535
gosh -- 1     probability= 0.0019120458891013384
lifeoh -- 1     probability= 0.0019120458891013384
relief -- 1     probability= 0.0019120458891013384
walk -- 1     probability= 0.0019120458891013384
theatre -- 1     probability= 0.0019120458891013384
minutes -- 7     probability= 0.01338432122370937
few -- 6     probability= 0.011472275334608031
hate -- 2     probability= 0.0038240917782026767
yeah -- 2     probability= 0.0038240917782026767
sucked -- 3     probability= 0.0057361376673040155
storyline -- 4     probability= 0.0076481835564053535
girlfriendboyfriend -- 1     probability= 0.0019120458891013384
pillow -- 1     probability= 0.0019120458891013384
keep -- 3     probability= 0.0057361376673040155
disliked -- 2     probability= 0.0038240917782026767
gone -- 1     probability= 0.0019120458891013384
dollars -- 1     probability= 0.0019120458891013384
tickets -- 1     probability

angel -- 1     probability= 0.0019120458891013384
yelps -- 1     probability= 0.0019120458891013384
specialtoo -- 1     probability= 0.0019120458891013384
explain -- 1     probability= 0.0019120458891013384
junkyard -- 1     probability= 0.0019120458891013384
funnyall -- 1     probability= 0.0019120458891013384
dogs -- 2     probability= 0.0038240917782026767
adorablethe -- 1     probability= 0.0019120458891013384
laughedkids -- 1     probability= 0.0019120458891013384
romanticcharminghilariousand -- 1     probability= 0.0019120458891013384
itbuy -- 1     probability= 0.0019120458891013384
comes -- 2     probability= 0.0038240917782026767
outit -- 1     probability= 0.0019120458891013384
chosen -- 1     probability= 0.0019120458891013384
american -- 1     probability= 0.0019120458891013384
propaganda -- 1     probability= 0.0019120458891013384
blatant -- 1     probability= 0.0019120458891013384
tortured -- 1     probability= 0.0019120458891013384
disgusting -- 1     probability= 0.0019

coppolas -- 1     probability= 0.0019120458891013384
costumes -- 1     probability= 0.0019120458891013384
crisp -- 1     probability= 0.0019120458891013384
baileys -- 1     probability= 0.0019120458891013384
eiko -- 1     probability= 0.0019120458891013384
ishioka -- 1     probability= 0.0019120458891013384
dracula -- 1     probability= 0.0019120458891013384
design -- 1     probability= 0.0019120458891013384
colored -- 1     probability= 0.0019120458891013384
went -- 1     probability= 0.0019120458891013384
beautifully -- 1     probability= 0.0019120458891013384
format -- 1     probability= 0.0019120458891013384
baaaaaad -- 1     probability= 0.0019120458891013384
allowing -- 1     probability= 0.0019120458891013384
miniseries -- 1     probability= 0.0019120458891013384
youre -- 1     probability= 0.0019120458891013384
unless -- 1     probability= 0.0019120458891013384
skip -- 1     probability= 0.0019120458891013384
collect -- 1     probability= 0.0019120458891013384
austens -- 1     

In [85]:
neg_words_freq.pop('', None)
neg_condi_prob={}
for word in neg_words_freq:
    print(word,"--",neg_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",neg_words_freq[word]/len(train_neg_reviews))
    neg_condi_prob[word]=neg_words_freq[word]/len(train_neg_reviews)

slowmoving -- 1       Conditional probability of slowmoving  given sentiment is neg= 0.003424657534246575
a -- 93       Conditional probability of a  given sentiment is neg= 0.3184931506849315
drifting -- 1       Conditional probability of drifting  given sentiment is neg= 0.003424657534246575
very -- 21       Conditional probability of very  given sentiment is neg= 0.07191780821917808
man -- 4       Conditional probability of man  given sentiment is neg= 0.0136986301369863
movie -- 52       Conditional probability of movie  given sentiment is neg= 0.1780821917808219
about -- 16       Conditional probability of about  given sentiment is neg= 0.0547945205479452
aimless -- 1       Conditional probability of aimless  given sentiment is neg= 0.003424657534246575
distressed -- 1       Conditional probability of distressed  given sentiment is neg= 0.003424657534246575
young -- 2       Conditional probability of young  given sentiment is neg= 0.00684931506849315
characters -- 10       Conditi

really -- 13       Conditional probability of really  given sentiment is neg= 0.04452054794520548
are -- 22       Conditional probability of are  given sentiment is neg= 0.07534246575342465
interested -- 1       Conditional probability of interested  given sentiment is neg= 0.003424657534246575
any -- 10       Conditional probability of any  given sentiment is neg= 0.03424657534246575
amazing -- 3       Conditional probability of amazing  given sentiment is neg= 0.010273972602739725
makers -- 1       Conditional probability of makers  given sentiment is neg= 0.003424657534246575
money -- 3       Conditional probability of money  given sentiment is neg= 0.010273972602739725
liked -- 1       Conditional probability of liked  given sentiment is neg= 0.003424657534246575
bluegreenscreen -- 1       Conditional probability of bluegreenscreen  given sentiment is neg= 0.003424657534246575
addition -- 2       Conditional probability of addition  given sentiment is neg= 0.00684931506849315
creat

mirrormask -- 1       Conditional probability of mirrormask  given sentiment is neg= 0.003424657534246575
dramatic -- 1       Conditional probability of dramatic  given sentiment is neg= 0.003424657534246575
inexperience -- 1       Conditional probability of inexperience  given sentiment is neg= 0.003424657534246575
meant -- 1       Conditional probability of meant  given sentiment is neg= 0.003424657534246575
passed -- 1       Conditional probability of passed  given sentiment is neg= 0.003424657534246575
unfortunately -- 3       Conditional probability of unfortunately  given sentiment is neg= 0.010273972602739725
conflict -- 1       Conditional probability of conflict  given sentiment is neg= 0.003424657534246575
direction -- 2       Conditional probability of direction  given sentiment is neg= 0.00684931506849315
way -- 9       Conditional probability of way  given sentiment is neg= 0.030821917808219176
tension -- 2       Conditional probability of tension  given sentiment is neg= 

acted -- 1       Conditional probability of acted  given sentiment is neg= 0.003424657534246575
remember -- 2       Conditional probability of remember  given sentiment is neg= 0.00684931506849315
eloquently -- 1       Conditional probability of eloquently  given sentiment is neg= 0.003424657534246575
woa -- 1       Conditional probability of woa  given sentiment is neg= 0.003424657534246575
believed -- 1       Conditional probability of believed  given sentiment is neg= 0.003424657534246575
favourite -- 1       Conditional probability of favourite  given sentiment is neg= 0.003424657534246575
first -- 5       Conditional probability of first  given sentiment is neg= 0.017123287671232876
talk -- 3       Conditional probability of talk  given sentiment is neg= 0.010273972602739725
either -- 1       Conditional probability of either  given sentiment is neg= 0.003424657534246575
surprisingly -- 2       Conditional probability of surprisingly  given sentiment is neg= 0.00684931506849315
ca

dry -- 1       Conditional probability of dry  given sentiment is neg= 0.003424657534246575
perfect -- 1       Conditional probability of perfect  given sentiment is neg= 0.003424657534246575
underneath -- 1       Conditional probability of underneath  given sentiment is neg= 0.003424657534246575
hockey -- 1       Conditional probability of hockey  given sentiment is neg= 0.003424657534246575
plants -- 1       Conditional probability of plants  given sentiment is neg= 0.003424657534246575
potted -- 1       Conditional probability of potted  given sentiment is neg= 0.003424657534246575
hes -- 1       Conditional probability of hes  given sentiment is neg= 0.003424657534246575
guy -- 1       Conditional probability of guy  given sentiment is neg= 0.003424657534246575
shameful -- 1       Conditional probability of shameful  given sentiment is neg= 0.003424657534246575
forgetting -- 1       Conditional probability of forgetting  given sentiment is neg= 0.003424657534246575
memorable -- 1  

turns -- 1       Conditional probability of turns  given sentiment is neg= 0.003424657534246575
unintentionally -- 1       Conditional probability of unintentionally  given sentiment is neg= 0.003424657534246575
widmark -- 1       Conditional probability of widmark  given sentiment is neg= 0.003424657534246575
marbles -- 1       Conditional probability of marbles  given sentiment is neg= 0.003424657534246575
early -- 1       Conditional probability of early  given sentiment is neg= 0.003424657534246575
constant -- 1       Conditional probability of constant  given sentiment is neg= 0.003424657534246575
indoor -- 1       Conditional probability of indoor  given sentiment is neg= 0.003424657534246575
exteriors -- 1       Conditional probability of exteriors  given sentiment is neg= 0.003424657534246575
isnt -- 1       Conditional probability of isnt  given sentiment is neg= 0.003424657534246575
studio -- 1       Conditional probability of studio  given sentiment is neg= 0.003424657534246

avoided -- 1       Conditional probability of avoided  given sentiment is neg= 0.003424657534246575
sum -- 1       Conditional probability of sum  given sentiment is neg= 0.003424657534246575
recommended -- 2       Conditional probability of recommended  given sentiment is neg= 0.00684931506849315
timing -- 1       Conditional probability of timing  given sentiment is neg= 0.003424657534246575
unbearable -- 1       Conditional probability of unbearable  given sentiment is neg= 0.003424657534246575
comedic -- 1       Conditional probability of comedic  given sentiment is neg= 0.003424657534246575
charisma -- 1       Conditional probability of charisma  given sentiment is neg= 0.003424657534246575
loads -- 1       Conditional probability of loads  given sentiment is neg= 0.003424657534246575
debits -- 1       Conditional probability of debits  given sentiment is neg= 0.003424657534246575
popcorn -- 1       Conditional probability of popcorn  given sentiment is neg= 0.003424657534246575
b

diabetic -- 1       Conditional probability of diabetic  given sentiment is neg= 0.003424657534246575
format -- 1       Conditional probability of format  given sentiment is neg= 0.003424657534246575
production -- 1       Conditional probability of production  given sentiment is neg= 0.003424657534246575
baaaaaad -- 1       Conditional probability of baaaaaad  given sentiment is neg= 0.003424657534246575
allowing -- 1       Conditional probability of allowing  given sentiment is neg= 0.003424657534246575
miniseries -- 1       Conditional probability of miniseries  given sentiment is neg= 0.003424657534246575
youre -- 1       Conditional probability of youre  given sentiment is neg= 0.003424657534246575
unless -- 1       Conditional probability of unless  given sentiment is neg= 0.003424657534246575
skip -- 1       Conditional probability of skip  given sentiment is neg= 0.003424657534246575
collect -- 1       Conditional probability of collect  given sentiment is neg= 0.003424657534246

In [86]:
neg_condi_prob

{'slowmoving': 0.003424657534246575,
 'a': 0.3184931506849315,
 'drifting': 0.003424657534246575,
 'very': 0.07191780821917808,
 'man': 0.0136986301369863,
 'movie': 0.1780821917808219,
 'about': 0.0547945205479452,
 'aimless': 0.003424657534246575,
 'distressed': 0.003424657534246575,
 'young': 0.00684931506849315,
 'characters': 0.03424657534246575,
 'half': 0.017123287671232876,
 'of': 0.2808219178082192,
 'was': 0.2089041095890411,
 'audience': 0.00684931506849315,
 'nearly': 0.003424657534246575,
 'or': 0.06164383561643835,
 'lost': 0.003424657534246575,
 'the': 0.5856164383561644,
 'walked': 0.003424657534246575,
 'out': 0.04794520547945205,
 'more': 0.030821917808219176,
 'not': 0.10273972602739725,
 'who': 0.05136986301369863,
 'whom': 0.00684931506849315,
 'flat': 0.00684931506849315,
 'sure': 0.010273972602739725,
 'plot': 0.0547945205479452,
 'nonexistent': 0.003424657534246575,
 'black': 0.0136986301369863,
 'even': 0.07876712328767123,
 'as': 0.08904109589041095,
 'poor': 

In [87]:
pos_words_freq.pop('', None)
pos_condi_prob={}
for word in pos_words_freq:
    print(word,"--",pos_words_freq[word],"      Conditional probability of",word," given sentiment is neg=",pos_words_freq[word]/len(train_pos_reviews))
    pos_condi_prob[word]=pos_words_freq[word]/len(train_pos_reviews)

a -- 101       Conditional probability of a  given sentiment is neg= 0.43722943722943725
scene -- 1       Conditional probability of scene  given sentiment is neg= 0.004329004329004329
movie -- 40       Conditional probability of movie  given sentiment is neg= 0.17316017316017315
to -- 56       Conditional probability of to  given sentiment is neg= 0.24242424242424243
find -- 4       Conditional probability of find  given sentiment is neg= 0.017316017316017316
in -- 45       Conditional probability of in  given sentiment is neg= 0.19480519480519481
best -- 10       Conditional probability of best  given sentiment is neg= 0.04329004329004329
was -- 37       Conditional probability of was  given sentiment is neg= 0.16017316017316016
head -- 1       Conditional probability of head  given sentiment is neg= 0.004329004329004329
gerardo -- 1       Conditional probability of gerardo  given sentiment is neg= 0.004329004329004329
keeps -- 1       Conditional probability of keeps  given sentimen

often -- 2       Conditional probability of often  given sentiment is neg= 0.008658008658008658
just -- 9       Conditional probability of just  given sentiment is neg= 0.03896103896103896
veteran -- 1       Conditional probability of veteran  given sentiment is neg= 0.004329004329004329
nostalgia -- 1       Conditional probability of nostalgia  given sentiment is neg= 0.004329004329004329
more -- 13       Conditional probability of more  given sentiment is neg= 0.05627705627705628
cast -- 9       Conditional probability of cast  given sentiment is neg= 0.03896103896103896
actors -- 5       Conditional probability of actors  given sentiment is neg= 0.021645021645021644
trip -- 1       Conditional probability of trip  given sentiment is neg= 0.004329004329004329
than -- 5       Conditional probability of than  given sentiment is neg= 0.021645021645021644
portrayal -- 2       Conditional probability of portrayal  given sentiment is neg= 0.008658008658008658
nuns -- 1       Conditional pr

screen -- 2       Conditional probability of screen  given sentiment is neg= 0.008658008658008658
age -- 2       Conditional probability of age  given sentiment is neg= 0.008658008658008658
gallon -- 1       Conditional probability of gallon  given sentiment is neg= 0.004329004329004329
punched -- 1       Conditional probability of punched  given sentiment is neg= 0.004329004329004329
isnt -- 1       Conditional probability of isnt  given sentiment is neg= 0.004329004329004329
jamie -- 1       Conditional probability of jamie  given sentiment is neg= 0.004329004329004329
spew -- 1       Conditional probability of spew  given sentiment is neg= 0.004329004329004329
quaid -- 1       Conditional probability of quaid  given sentiment is neg= 0.004329004329004329
noteworthy -- 2       Conditional probability of noteworthy  given sentiment is neg= 0.008658008658008658
received -- 2       Conditional probability of received  given sentiment is neg= 0.008658008658008658
another -- 2       Condi

feisty -- 1       Conditional probability of feisty  given sentiment is neg= 0.004329004329004329
roosevelts -- 1       Conditional probability of roosevelts  given sentiment is neg= 0.004329004329004329
ruthless -- 1       Conditional probability of ruthless  given sentiment is neg= 0.004329004329004329
lieutenant -- 1       Conditional probability of lieutenant  given sentiment is neg= 0.004329004329004329
son -- 1       Conditional probability of son  given sentiment is neg= 0.004329004329004329
kanalys -- 1       Conditional probability of kanalys  given sentiment is neg= 0.004329004329004329
member -- 1       Conditional probability of member  given sentiment is neg= 0.004329004329004329
spiffy -- 1       Conditional probability of spiffy  given sentiment is neg= 0.004329004329004329
hay -- 1       Conditional probability of hay  given sentiment is neg= 0.004329004329004329
radiant -- 1       Conditional probability of radiant  given sentiment is neg= 0.004329004329004329
steve --

amusing -- 1       Conditional probability of amusing  given sentiment is neg= 0.004329004329004329
jobs -- 1       Conditional probability of jobs  given sentiment is neg= 0.004329004329004329
last -- 1       Conditional probability of last  given sentiment is neg= 0.004329004329004329
minutes -- 1       Conditional probability of minutes  given sentiment is neg= 0.004329004329004329
assante -- 1       Conditional probability of assante  given sentiment is neg= 0.004329004329004329
sounded -- 1       Conditional probability of sounded  given sentiment is neg= 0.004329004329004329
watched -- 2       Conditional probability of watched  given sentiment is neg= 0.008658008658008658
armand -- 1       Conditional probability of armand  given sentiment is neg= 0.004329004329004329
companys -- 1       Conditional probability of companys  given sentiment is neg= 0.004329004329004329
summary -- 1       Conditional probability of summary  given sentiment is neg= 0.004329004329004329
already -- 1

tons -- 1       Conditional probability of tons  given sentiment is neg= 0.004329004329004329
crowd -- 1       Conditional probability of crowd  given sentiment is neg= 0.004329004329004329
effects -- 3       Conditional probability of effects  given sentiment is neg= 0.012987012987012988
yet -- 2       Conditional probability of yet  given sentiment is neg= 0.008658008658008658
hear -- 1       Conditional probability of hear  given sentiment is neg= 0.004329004329004329
amazed -- 1       Conditional probability of amazed  given sentiment is neg= 0.004329004329004329
timeless -- 1       Conditional probability of timeless  given sentiment is neg= 0.004329004329004329
imaginative -- 1       Conditional probability of imaginative  given sentiment is neg= 0.004329004329004329
turkey -- 1       Conditional probability of turkey  given sentiment is neg= 0.004329004329004329
cruel -- 1       Conditional probability of cruel  given sentiment is neg= 0.004329004329004329
straw -- 1       Condi

shot -- 1       Conditional probability of shot  given sentiment is neg= 0.004329004329004329
california -- 1       Conditional probability of california  given sentiment is neg= 0.004329004329004329
using -- 1       Conditional probability of using  given sentiment is neg= 0.004329004329004329
documentary -- 1       Conditional probability of documentary  given sentiment is neg= 0.004329004329004329
desert -- 1       Conditional probability of desert  given sentiment is neg= 0.004329004329004329
watkins -- 2       Conditional probability of watkins  given sentiment is neg= 0.008658008658008658
patent -- 1       Conditional probability of patent  given sentiment is neg= 0.004329004329004329
southern -- 1       Conditional probability of southern  given sentiment is neg= 0.004329004329004329
accessible -- 1       Conditional probability of accessible  given sentiment is neg= 0.004329004329004329
peter -- 1       Conditional probability of peter  given sentiment is neg= 0.004329004329004

In [88]:
pos_condi_prob

{'a': 0.43722943722943725,
 'scene': 0.004329004329004329,
 'movie': 0.17316017316017315,
 'to': 0.24242424242424243,
 'find': 0.017316017316017316,
 'in': 0.19480519480519481,
 'best': 0.04329004329004329,
 'was': 0.16017316017316016,
 'head': 0.004329004329004329,
 'gerardo': 0.004329004329004329,
 'keeps': 0.004329004329004329,
 'that': 0.12987012987012986,
 'trying': 0.004329004329004329,
 'his': 0.04329004329004329,
 'is': 0.2857142857142857,
 'song': 0.008658008658008658,
 'the': 0.5974025974025974,
 'through': 0.012987012987012988,
 'when': 0.030303030303030304,
 'running': 0.004329004329004329,
 'good': 0.06926406926406926,
 'effort': 0.004329004329004329,
 'kids': 0.008658008658008658,
 'and': 0.45454545454545453,
 'it': 0.24242424242424243,
 'messages': 0.004329004329004329,
 'thought': 0.012987012987012988,
 'saw': 0.017316017316017316,
 'for': 0.1038961038961039,
 'today': 0.008658008658008658,
 'casting': 0.017316017316017316,
 'as': 0.09090909090909091,
 'loved': 0.017316

In [89]:
def score(real, prediction):
    count = 0
    for i in range(len(real)):
        if real[i] == prediction[i]:
            count+=1
    accuracy=count/len()
    return correct / float(len(actual)) * 100.0

In [90]:
def pred(data,ans):
    y_preds=[]
    for sentence in data:
        sentence=text_filtering(sentence)
        pos_lik=1
        neg_lik=1
        for word in sentence.split(' '):
            word=word.lower()
            if word not in pos_condi_prob.keys():
                pos_condi_prob[word]=0
            if word not in neg_condi_prob.keys():
                neg_condi_prob[word]=0
            pos_lik*=pos_condi_prob[word]
            neg_lik*=neg_condi_prob[word]
        pos_posterior=pos_lik*prior_probs[1]
        neg_posterior=neg_lik*prior_probs[0]
        if pos_posterior>neg_posterior:
            y_preds.append(1)
        else:
            y_preds.append(0)
    
    return score(ans.tolist(),y_preds)
print("Accuracy on training data--",pred(X_train,y_train) )   
print("Accuracy on development data--",pred(X_dev,y_dev))
print("Accuracy on test data--",pred(X_test,y_test))

Accuracy on training data-- 93.30783938814533
Accuracy on development data-- 28.57142857142857
Accuracy on test data-- 38.93805309734513
